In [53]:
import pickle
with open('./train_data', 'rb') as f:
    train_data = pickle.load(f)
    train_label= pickle.load(f)
with open('./test_data', 'rb') as f:
    test_data = pickle.load(f)

In [63]:
print(train_data.shape)
print(len(train_data))
train_data = train_data.reshape((len(train_data), 3, 32, 32)).transpose(0, 2, 3, 1)
print(train_data.shape)
test_data = test_data.reshape((len(test_data), 3, 32, 32)).transpose(0, 2, 3, 1)
print(test_data.shape)

(50000, 3072)
50000
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [70]:
print(test_data.shape[0])

10000


In [64]:
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers

In [65]:
class cifar100vgg:
    def __init__(self,train=True):
        self.num_classes = 100
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar100vgg.h5')
            
    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.
        model = Sequential()
        weight_decay = self.weight_decay
        model.add(Conv2D(64, (3, 3), padding='same',input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        
        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        model.summary()
        return model
    
    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 2
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20

        # The data, shuffled and split between train and test sets:
        # (x_train, y_train), (x_test, y_test) = cifar100.load_data()
        x_train = train_data.astype('float32')
        print(train_data.shape)
        print(x_train.shape)
        x_test = test_data.astype('float32')
        # x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(train_label, self.num_classes)
        #y_test = keras.utils.to_categorical(y_test, self.num_classes)


        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        #datagen.fit(x_train)
        
        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        '''historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)'''
        historytemp = model.fit(x_train, y_train, batch_size=batch_size, epochs= 2)
        # model.save_weights('cifar100vgg.h5')
        return model
    
    def predict(self,x,normalize=True,batch_size=50):
        '''if normalize:
            x = self.normalize_production(x)'''
        return self.model.predict(x,batch_size)

In [66]:
if __name__ == '__main__':
    #(x_train, y_train), (x_test, y_test) = cifar100.load_data()
    #x_train = train_data.astype('float32')
    x_test = test_data.astype('float32')

    #y_train = keras.utils.to_categorical(y_train, 100)
    #y_test = keras.utils.to_categorical(y_test, 100)

    model = cifar100vgg()

    #predicted_x = model.predict(x_test)
    #residuals = (np.argmax(predicted_x,1)!=np.argmax(y_test,1))
    #loss = sum(residuals)/len(residuals)
    #print("the validation 0/1 loss is: ",loss)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_58 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_39 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_39 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 65536)             0         
_________________________________________________________________
dense_39 (Dense)             (None, 512)               33554944  
_________________________________________________________________
activation_59 (Activation)   (None, 512)               0         
__________